# Projeto Final - Sistema de Controle Financeiro

Deverá ser desenvolvido um sistema para controle financeiro que receba as movimentações e as armazena em um arquivo csv ou json.

O sistema deverá ser capaz de realizar as seguintes operações:

- **Criar** novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor.

- Os tipos podem ser despesas, receita ou investimento:
  - No caso de receita, o valor deve ser tratado como numerico e armazenado normalmente.
  - no caso de despesas o valor deve ser recebido como positivo, mas armazenado como negativo
  - No caso de investimento, deve ter uma informação a mais de 'Montante', em que será calculado quanto o dinheiro rendeu desde o dia que foi investido. Para essa finalidade utilize a seguinte formula: $M = C * (1 + i)^t$ ([Saiba mais](https://matematicafinanceira.org/juros-compostos/)), considere tudo em dias.
- **Ler** registros: Deverá ser possível consultar os registros por data, tipo ou valor.
- **Atualizar** registros: No caso de atualização, pode-se atualizar o valor, o tipo e a data deverá ser a de atualização do registro.
- **Deletar**: Deverá ser possível deletar o registro (caso necessário, considere o indice do elemento como ID)
- Crie uma função que atualize os valores de rendimento sempre que chamada
- Crie uma função exportar_relatorio, que seja possível exportar um relatorio final em csv ou json .
- Crie pelo menos uma função de agrupamento, que seja capaz de mostrar o total de valor baseado em alguma informação (mes, tipo...)
- Crie valores separados para identificar a data (dia, mes, ano)

---

Atenção:
- Não utilize a biblioteca pandas para resolução desse exercício
- Deem um nome criativo para a aplicação de vocês

In [ ]:
import csv
import json
from datetime import datetime

# import uuid
from datetime import datetime
from datetime import timezone
from datetime import timedelta



def salvar_em_json(registros, nome_salvar):
    with open(nome_salvar, 'w') as arq_json:
        json.dump(registros, arq_json)

def ler_em_json(nome_salvar):
    try:
        with open(nome_salvar, 'r') as arq_json:
            dados_json = json.load(arq_json)
        return dados_json
    except FileNotFoundError:
        return []
   

    
def retorna_hoje(offset:int=-3)->datetime.date:
    """
    Essa funcao retorna a data de hoje no fuso horario informado. 
    Por padrão o fuso horario é o UTC -3:00, America/Sao_Paulo
    """
    timezone_offset = offset  # America/Sao_Paulo (UTC−03:00)
    tzinfo = timezone(timedelta(hours=timezone_offset))
    today = datetime.now(tzinfo).date()
    return today



def adicionar_registro(id_usuario):
    
    registros = ler_em_json('registros.json')
    
    data = str(retorna_hoje())
    tipo = input('Digite o tipo do registro Despesa: d, Receita: r, Investimento: i, Sair: s\n')
    
    if tipo == 's':
        return        
    elif tipo == 'd':
        tipo = "Despesa"
    elif tipo == 'r':
        tipo = "Receita"
    elif tipo == 'i':
        tipo = "Investimento"
    else:
        print('Tipo inválido. Saindo...')
        return
        
        
    valor = float(input(f'digite o valor do(a) {tipo}: '))
    categoria = input(f'digite a categoria do(a) {tipo}: ')
    
    try:
        id_registro = max([i['id_registro'] for i in registros]) + 1
    except:
        id_registro = 0
        
    
    # Separar a data em dia, mês e ano
    ano, mes, dia = map(int, data.split('-'))

    # Tratar despesas como valores negativos
    if tipo == 'Despesa':
        valor = -valor

    # Adicionar novo registro à lista
    novo_registro = {'id_registro': id_registro, 'id_usuario': id_usuario, 'tipo': tipo, 'categoria': categoria, 'valor': valor, 'dia': dia, 'mes': mes, 'ano': ano}
    registros.append(novo_registro)
    salvar_em_json(registros,'registros.json')



def ler_registro(id_usuario):
    registros = ler_em_json('registros.json')
    consulta_registros = [registro for registro in registros if registro['id_usuario'] == id_usuario]
    
    print(f"\nRegistros do usuário {id_usuario} ")
    
    print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<5} | {:<5} | {:<5} |".format(
        "ID", "Tipo", "Categoria", "Valor", "Dia", "Mês", "Ano"
    ))
    print("-" * 76)
    
    for registro in consulta_registros:
        print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<5} | {:<5} | {:<5} |".format(
            registro['id_registro'],
            registro['tipo'],
            registro['categoria'],
            registro['valor'],
            registro['dia'],
            registro['mes'],
            registro['ano']
        ))
    
    print("-" * 76)
        
        
def editar_registro(id_usuario):
    
    data = str(retorna_hoje())
    ano, mes, dia = map(int, data.split('-'))
    
    registros = ler_em_json('registros.json')
    consulta_registros = [registro for registro in registros if registro['id_usuario'] == id_usuario]    
    print(f"\nRegistros do usuário {id_usuario} ")
    
    
    print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<10} | {:<10} | {:<10} ".format(
        "ID", "Tipo", "Categoria", "Valor", "Dia", "Mês","Ano"
    ))
    print("-" * 76)
    
    for registro in consulta_registros:
        
        print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<10} | {:<10} |".format(
            registro['id_registro'],
            registro['tipo'],
            registro['categoria'],
            registro['valor'],
            registro['dia'],
            f"{registro['dia']}/{registro['mes']}/{registro['ano']}"
        ))
        
    id_registro_para_editar = int(input("Informe o ID do REGISTRO que deseja editar "))    
    indice_para_editar = [i for i, registro in enumerate(registros) if registro['id_registro'] == id_registro_para_editar]    
    
    tipo = input('Digite o novo TIPO do registro - Despesa: d, Receita: r, Investimento: i\n')
    valor = float(input('Digite o novo VALOR do registro\n'))
    
                        
    if tipo == 'd':
        tipo = "Despesa"
    elif tipo == 'r':
        tipo = "Receita"
    elif tipo == 'i':
        tipo = "Investimento"
    else:
        print('Opção inválida. Saindo...')
        return
    
    # Tratar despesas como valores negativos
    if tipo == 'Despesa':
        valor = -valor
        
    registros[indice_para_editar[0]]['valor'] = valor
    registros[indice_para_editar[0]]['tipo'] = tipo
    registros[indice_para_editar[0]]['dia'] = dia
    registros[indice_para_editar[0]]['mes'] = mes
    registros[indice_para_editar[0]]['ano'] = ano

    salvar_em_json(registros,'registros.json')

    print("Registro Editado!")
        
    
    
def deletar_registro(id_usuario):    
    registros = ler_em_json('registros.json')
    registros_do_usuario = [registro for registro in registros if registro['id_usuario'] == id_usuario]
    
    print(f"\nRegistros do usuário {id_usuario} ")
    
    
    print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<10} | {:<10} |".format(
        "ID", "Tipo", "Categoria", "Valor", "Dia", "Data"
    ))
    print("-" * 76)
    
    for registro in registros_do_usuario:
        
        print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<10} | {:<10} |".format(
            registro['id_registro'],
            registro['tipo'],
            registro['categoria'],
            registro['valor'],
            registro['dia'],
            f"{registro['dia']}/{registro['mes']}/{registro['ano']}"
        ))
        
    id_registro_para_deletar = int(input("Informe o ID do REGISTRO que deseja deletar "))
    
    escolha = input(f'Tem certeza que deseja DELETAR o registro {id_registro_para_deletar}? S/N ')
    
    if escolha.upper() == 'S':
        registros = [registro for registro in registros if registro['id_registro'] != id_registro_para_deletar]

        salvar_em_json(registros,'registros.json')

        print("Registro Deletado!")
        
    elif escolha.upper() == 'N':
        print("Operação Cancelada ")
        
    else:
        print("Opção Inválida ")
        print("Voltando para o menu principal ")
        
def filtrar_registro(id_usuario):
    registros = ler_em_json('registros.json')
    
    filtro_tipo = input("Digite o tipo de filtro (dia, mês ou ano): ").lower()
    filtro_valor = int(input(f"Digite o valor do filtro para o(a) {filtro_tipo}: "))
    
    consulta_registros1 = []

    if filtro_tipo == 'dia':
        consulta_registros1 = [registro for registro in registros if registro['id_usuario'] == id_usuario and registro['dia'] == filtro_valor]
    elif filtro_tipo == 'mês':
        consulta_registros1 = [registro for registro in registros if registro['id_usuario'] == id_usuario and registro['mes'] == filtro_valor]
    elif filtro_tipo == 'ano':
        consulta_registros1 = [registro for registro in registros if registro['id_usuario'] == id_usuario and registro['ano'] == filtro_valor]
    else:
        print("Tipo de filtro inválido. Voltando para o menu principal.")
        return
    
    print(f"\nRegistros do usuário {id_usuario} filtrados pelo {filtro_tipo} {filtro_valor}")
    
    print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<5} | {:<5} | {:<5} |".format(
        "ID", "Tipo", "Categoria", "Valor", "Dia", "Mês", "Ano"
    ))
    print("-" * 76)
    
    for registro in consulta_registros1:
        print("| {:<10} | {:<15} | {:<15} | {:<10} | {:<5} | {:<5} | {:<5} |".format(
            registro['id_registro'],
            registro['tipo'],
            registro['categoria'],
            registro['valor'],
            registro['dia'],
            registro['mes'],
            registro['ano']
        ))
    
    print("-" * 76)
    
    
def menu_principal():    
    
    menu = int(input(' 1 - Criar novo Registro\n 2 - Visualizar Registro\n 3 - Editar Registro\n 4 - Deletar Registro\n 5 - Visualizar por Filtro (dia,mês,ano)\n 0 - Sair do Programa\n O que deseja fazer? (utilize os índices para utilizar o menu) '))

    if menu == 1: #Adicionar registro
        adicionar_registro(id_usuario)
        
    elif menu == 2: #Visualizar registro
        ler_registro(id_usuario)
            
    elif menu == 3: #Editar registro
        editar_registro(id_usuario)
        
    elif menu == 4: #Deletar registro
        deletar_registro(id_usuario)
        
    elif menu == 5:  # Filtrar por Dia/Mês/Ano
        filtrar_registro(id_usuario)
        
    elif menu == 0:
        return 0
        
    else:
        print('Opção Inválida')

        
id_usuario = input("Informe o ID do Usuário: ")    

while menu_principal() != 0:
    menu_principal()


Informe o ID do Usuário: 444
 1 - Criar novo Registro
 2 - Visualizar Registro
 3 - Editar Registro
 4 - Deletar Registro
 5 - Visualizar por Filtro (dia,mês,ano)
 0 - Sair do Programa
 O que deseja fazer? (utilize os índices para utilizar o menu) 2

Registros do usuário 444 
| ID         | Tipo            | Categoria       | Valor      | Dia   | Mês   | Ano   |
----------------------------------------------------------------------------
| 3          | Receita         | forró           | 150.0      | 22    | 11    | 2023  |
| 4          | Investimento    | jogo do bicho   | 250.0      | 22    | 11    | 2023  |
| 6          | Despesa         | roça            | -150.0     | 22    | 11    | 2023  |
----------------------------------------------------------------------------
